In [5]:
def generate_insitu_stress(group_name, sz_value, grad_z_sx, grad_z_sy, grad_z_sz, grad_origin="0,0,0"):
    """
    生成 3DEC 的 block insitu stress 命令，动态计算 x=z/2, y=z/2。
    
    参数:
        group_name (str): 块体组名
        sz_value (float): 初始垂向应力分量 (z方向)
        grad_z_sx (float): 垂向梯度对水平应力x的影响系数
        grad_z_sy (float): 垂向梯度对水平应力y的影响系数
        grad_z_sz (float): 垂向梯度对垂向应力z的影响系数
        grad_origin (str): 梯度原点（默认为 "0,0,0"）
    
    返回:
        str: 完整的 3DEC 命令
    """
    sx = sz_value / 2
    sy = sz_value / 2
    command = (
        f"block insitu stress {sx} {sy} {sz_value} 0,0,0 "
        f"gradient-z {grad_z_sx} {grad_z_sy} {grad_z_sz} {grad_origin} "
        f"range group '{group_name}'"
    )
    return command

# 定义参数列表（每组对应一个字典）
params = [
    {"group": "1", "sz": 0, "grad_z": (13000, 13000, 26000)},
    {"group": "2", "sz": -120680, "grad_z": (12650, 12650, 25300)},
    {"group": "3", "sz": 35000, "grad_z": (13000, 13000, 26000)},
    {"group": "4", "sz": -27480, "grad_z": (12900, 12900, 25800)},
    {"group": "5", "sz": 52400, "grad_z": (13000, 13000, 26000)},
    {"group": "6", "sz": 52400, "grad_z": (13000, 13000, 26000)},
    {"group": "7", "sz": -241180, "grad_z": (12650, 12650, 25300)},
    {"group": "8", "sz": 58000, "grad_z": (13000, 13000, 26000)},
    {"group": "9", "sz": -5130800, "grad_z": (7000, 7000, 14000)},
    {"group": "10", "sz": -5130800, "grad_z": (7000, 7000, 14000)},
    {"group": "11", "sz": 32880, "grad_z": (12900, 12900, 25800)},
]

# 批量生成命令
commands = []
for param in params:
    group = param["group"]
    sz = param["sz"]
    grad_z = param["grad_z"]
    cmd = generate_insitu_stress(
        group_name=group,
        sz_value=sz,
        grad_z_sx=grad_z[0],
        grad_z_sy=grad_z[1],
        grad_z_sz=grad_z[2]
    )
    commands.append(cmd)

# 打印所有命令
for cmd in commands:
    print(cmd)

block insitu stress 0.0 0.0 0 0,0,0 gradient-z 13000 13000 26000 0,0,0 range group '1'
block insitu stress -60340.0 -60340.0 -120680 0,0,0 gradient-z 12650 12650 25300 0,0,0 range group '2'
block insitu stress 17500.0 17500.0 35000 0,0,0 gradient-z 13000 13000 26000 0,0,0 range group '3'
block insitu stress -13740.0 -13740.0 -27480 0,0,0 gradient-z 12900 12900 25800 0,0,0 range group '4'
block insitu stress 26200.0 26200.0 52400 0,0,0 gradient-z 13000 13000 26000 0,0,0 range group '5'
block insitu stress 26200.0 26200.0 52400 0,0,0 gradient-z 13000 13000 26000 0,0,0 range group '6'
block insitu stress -120590.0 -120590.0 -241180 0,0,0 gradient-z 12650 12650 25300 0,0,0 range group '7'
block insitu stress 29000.0 29000.0 58000 0,0,0 gradient-z 13000 13000 26000 0,0,0 range group '8'
block insitu stress -2565400.0 -2565400.0 -5130800 0,0,0 gradient-z 7000 7000 14000 0,0,0 range group '9'
block insitu stress -2565400.0 -2565400.0 -5130800 0,0,0 gradient-z 7000 7000 14000 0,0,0 range group

In [8]:
import xlwings as xw
import pandas as pd

def update_c2_and_read_n(c2_values, input_path, output_path=None):
    app = xw.App(visible=False)
    results = []
    try:
        wb = app.books.open(input_path)
        sheet = wb.sheets["煤层融合"]
        
        for c2_value in c2_values:
            sheet.range("C2").value = c2_value
            wb.app.calculate()
            
            # 读取 N2:N12 并转换为整数
            n_values = sheet.range("N2:N12").value
            n_values_int = [int(round(val)) if val is not None else None for val in n_values]  # 确保处理空值
            
            results.append({
                "C2": c2_value,
                "iniszz": n_values_int
            })
            
            if output_path:
                wb.save(output_path.format(c2_value))
                
        return pd.DataFrame(results)
    
    finally:
        wb.close()
        app.quit()

# 使用示例
if __name__ == "__main__":
    c2_test_values = [550, 560, 570]
    df_results = update_c2_and_read_n(c2_test_values, "3deca.xlsx")
    
    # 展开 iniszz 列到多列（按行显示）
    df_expanded = df_results.explode("iniszz").reset_index(drop=True)
    df_expanded["Row"] = ["N2", "N3", "N4", "N5", "N6", "N7", "N8", "N9", "N10", "N11", "N12"] * len(c2_test_values)
    
    # 按 C2 值分组展示
    final_output = df_expanded.pivot(index="Row", columns="C2", values="iniszz")
    print(final_output)

C2        550       560       570
Row                              
N10  -3330800  -3450800  -3570800
N11  -3330800  -3450800  -3570800
N12     62880     60880     58880
N2          0         0         0
N3     -15680    -22680    -29680
N4      35000     35000     35000
N5       2520       520     -1480
N6      52400     52400     52400
N7      52400     52400     52400
N8    -136180   -143180   -150180
N9      58000     58000     58000


In [11]:
import xlwings as xw
import pandas as pd

# ======================== 核心函数 ========================
def process_c2_and_generate_commands(c2_values, input_excel_path):
    """
    完整处理流程：
    1. 修改 C2 值并读取 N 列整数
    2. 更新 params 参数表
    3. 生成 3DEC 命令
    """
    # ------------------------------------------
    # 步骤1: 读取 Excel 并获取 N 列整数数据
    # ------------------------------------------
    def read_and_update_excel(c2_value):
        """修改 C2 值并返回 N 列整数"""
        app = xw.App(visible=False)
        try:
            wb = app.books.open(input_excel_path)
            sheet = wb.sheets["煤层融合"]
            
            # 修改 C2 值并强制计算
            sheet.range("C2").value = c2_value
            wb.app.calculate()
            
            # 读取 N2-N12 并转换为整数
            n_values = sheet.range("N2:N12").value
            n_values_int = [int(round(val)) if val is not None else 0 for val in n_values]
            return n_values_int
        finally:
            wb.close()
            app.quit()

    # ------------------------------------------
    # 步骤2: 定义参数模板 (保留原始 grad_z 参数)
    # ------------------------------------------
    params_template = [
        {"group": "1", "grad_z": (13000, 13000, 26000)},
        {"group": "2", "grad_z": (12650, 12650, 25300)},
        {"group": "3", "grad_z": (13000, 13000, 26000)},
        {"group": "4", "grad_z": (12900, 12900, 25800)},
        {"group": "5", "grad_z": (13000, 13000, 26000)},
        {"group": "6", "grad_z": (13000, 13000, 26000)},
        {"group": "7", "grad_z": (12650, 12650, 25300)},
        {"group": "8", "grad_z": (13000, 13000, 26000)},
        {"group": "9", "grad_z": (7000, 7000, 14000)},
        {"group": "10", "grad_z": (7000, 7000, 14000)},
        {"group": "11", "grad_z": (12900, 12900, 25800)},
    ]
    
    # ------------------------------------------
    # 步骤3: 处理每个 C2 值并生成命令
    # ------------------------------------------
    all_commands = {}
    for c2_value in c2_values:
        # 获取 N 列数据
        n_values = read_and_update_excel(c2_value)
        
        # 检查数据长度
        if len(n_values) != len(params_template):
            raise ValueError(f"N列数据长度{len(n_values)}与参数组数{len(params_template)}不匹配")
        
        # 动态构建 params
        params = [
            {"group": item["group"], "sz": n_values[i], "grad_z": item["grad_z"]}
            for i, item in enumerate(params_template)
        ]
        
        # 生成命令
        commands = []
        for param in params:
            cmd = generate_insitu_stress(
                group_name=param["group"],
                sz_value=param["sz"],
                grad_z_sx=param["grad_z"][0],
                grad_z_sy=param["grad_z"][1],
                grad_z_sz=param["grad_z"][2]
            )
            commands.append(cmd)
        
        all_commands[c2_value] = commands
    
    return all_commands


def generate_insitu_stress(group_name, sz_value, grad_z_sx, grad_z_sy, grad_z_sz, grad_origin="0,0,0"):
    """生成 3DEC 的 block insitu stress 命令"""
    sx = sz_value / 2
    sy = sz_value / 2
    command = (
        f"block insitu stress {sx} {sy} {sz_value} 0,0,0 "
        f"gradient-z {grad_z_sx} {grad_z_sy} {grad_z_sz} {grad_origin} "
        f"range group '{group_name}'"
    )
    return command


# ======================== 使用示例 ========================
if __name__ == "__main__":
    # 定义要测试的 C2 值
    c2_test_values = [600]  # 可扩展更多值
    
    # 执行处理流程
    results = process_c2_and_generate_commands(
        c2_values=c2_test_values,
        input_excel_path="3deca.xlsx"
    )
    
    # 打印输出
    for c2_value, commands in results.items():
        print(f"\n====== C2 = {c2_value} 生成的命令 ======")
        for cmd in commands:
            print(cmd)


====== C2 = 600 生成的命令 ======
block insitu stress 0.0 0.0 0 0,0,0 gradient-z 13000 13000 26000 0,0,0 range group '1'
block insitu stress -25340.0 -25340.0 -50680 0,0,0 gradient-z 12650 12650 25300 0,0,0 range group '2'
block insitu stress 17500.0 17500.0 35000 0,0,0 gradient-z 13000 13000 26000 0,0,0 range group '3'
block insitu stress -3740.0 -3740.0 -7480 0,0,0 gradient-z 12900 12900 25800 0,0,0 range group '4'
block insitu stress 26200.0 26200.0 52400 0,0,0 gradient-z 13000 13000 26000 0,0,0 range group '5'
block insitu stress 26200.0 26200.0 52400 0,0,0 gradient-z 13000 13000 26000 0,0,0 range group '6'
block insitu stress -85590.0 -85590.0 -171180 0,0,0 gradient-z 12650 12650 25300 0,0,0 range group '7'
block insitu stress 29000.0 29000.0 58000 0,0,0 gradient-z 13000 13000 26000 0,0,0 range group '8'
block insitu stress -1965400.0 -1965400.0 -3930800 0,0,0 gradient-z 7000 7000 14000 0,0,0 range group '9'
block insitu stress -1965400.0 -1965400.0 -3930800 0,0,0 gradient-z 7000 7000